In [2]:
import torch
from torch.utils.data import DataLoader
from neuralop.models import GINO
from tensorly import tenalg
from cfd_dataset import CFDDataset, CFDDatasetWrapper

tenalg.set_backend("einsum")

def test_gino_single_batch(model, x, y, input_geom, latent_queries, output_queries, device):
    model.to(device)
    criterion = torch.nn.MSELoss()
    
    model.train()
    
    # Get a single batch from the train_loader
    # try:
    #     x, y, output_queries = next(iter(train_loader))
    # except StopIteration:
    #     print("Error: The train_loader is empty. Make sure your dataset is not empty.")
    #     return

    x, y = x.to(device), y.to(device)
    output_queries = output_queries.to(device)
    
    x.requires_grad_(True)  # Enable gradient computation for x
    
    print(f"x shape: {x.shape}")
    print(f"y shape: {y.shape}")
    print(f"input_geom shape: {input_geom.shape}")
    print(f"latent_queries shape: {latent_queries.shape}")
    print(f"output_queries shape: {output_queries.shape}")

    ada_in = torch.randn(1, device=device)  # Random ada_in for the batch
    
    # Forward pass
    output = model(x, input_geom, latent_queries, output_queries, ada_in=ada_in)
    
    # Compute loss
    loss = criterion(output, y)
    
    print(f"Loss: {loss.item()}")

    # Backward pass
    loss.backward()

    # Check if all parameters have gradients
    n_unused_params = sum(1 for param in model.parameters() if param.grad is None)
    print(f"Number of unused parameters: {n_unused_params}")

    # Check if output is finite
    if not output.isfinite().all():
        print("Warning: Output contains non-finite values!")

    if x.shape[0] > 1:
        # Check if x[1:] accumulates no grad
        if x.grad[1:].nonzero().any():
            print("Warning: Unexpected gradients in x[1:]")

    print("Single batch forward and backward pass completed.")

def main():
    # Hyperparameters
    batch_size = 2
    num_train_samples = 250  # Adjust based on your dataset

    # Create dataset
    train_dataset = CFDDataset('data/', num_train_samples)

    # Wrap dataset to only return x, y, and output_queries
    train_dataset_wrapped = CFDDatasetWrapper(train_dataset)

    # Create data loader
    train_loader = DataLoader(train_dataset_wrapped, batch_size=batch_size, shuffle=True, num_workers=1)

    # Get input_geom and latent_queries from the first item of the dataset
    x, y, input_geom, latent_queries, output_queries = train_dataset[0]

    # Initialize GINO model
    model = GINO(
        in_channels=1,  # 1 for pressure
        out_channels=1,  # 1 for smoke concentration
        gno_coord_dim=3,
        gno_radius=0.3,
        projection_channels=16,
        in_gno_mlp_hidden_layers=[16, 16],
        out_gno_mlp_hidden_layers=[16, 16],
        in_gno_transform_type="nonlinear",
        out_gno_transform_type="nonlinear",
        fno_n_modes=(16, 16, 16),
        fno_hidden_channels=64,
        fno_lifting_channels=16,
        fno_projection_channels=16,
        fno_norm="ada_in",
    )

    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Move input_geom and latent_queries to the device
    input_geom = input_geom.to(device)
    latent_queries = latent_queries.to(device)
        
    # Run the single batch test
    test_gino_single_batch(model, x, y, input_geom, latent_queries, output_queries, device)

if __name__ == '__main__':
    main()

Using device: cpu
x shape: torch.Size([1, 512, 1])
y shape: torch.Size([1, 512, 1])
input_geom shape: torch.Size([1, 512, 3])
latent_queries shape: torch.Size([1, 8, 8, 8, 3])
output_queries shape: torch.Size([512, 3])
latent_embed.shape=torch.Size([1, 64, 8, 8, 8])
latent_embed.shape=torch.Size([1, 512, 64]) after permute
---going into output GNO---
out.shape=torch.Size([1, 512, 64])
Loss: 0.030690349638462067
Number of unused parameters: 0
Single batch forward and backward pass completed.
